<a href="https://colab.research.google.com/github/John-auw/ML100-Days/blob/master/homework/Day_019_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

In [2]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

import requests
data_path = './'
url = 'http://ai100.cupoy.com/file-download/part02/titanic_train.csv'
r = requests.get(url, allow_redirects=True)
open('titanic_train.csv', 'wb').write(r.content)

url = 'http://ai100.cupoy.com/file-download/part02/titanic_test.csv'
r = requests.get(url, allow_redirects=True)
open('titanic_test.csv', 'wb').write(r.content)

df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [4]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
df.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [11]:
# 空值補 -1, 做羅吉斯迴歸
df_m1 = df.fillna(-1)
train_X = df_m1[:train_num]
estimator = LogisticRegression(solver='lbfgs')
_ = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print(f'空值補 -1, 做羅吉斯迴歸: {_:.3f}')
# 空值補 0, 做羅吉斯迴歸
df_0 = df.fillna(0)
train_X = df_0[:train_num]
estimator = LogisticRegression(solver='lbfgs')
_ = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print(f'空值補 0, 做羅吉斯迴歸: {_:.3f}')

# 空值補 median, 做羅吉斯迴歸
df_md = df.fillna(df.median())
train_X = df_md[:train_num]
estimator = LogisticRegression(solver='lbfgs')
_ = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print(f'空值補 median, 做羅吉斯迴歸: {_:.3f}')

空值補 -1, 做羅吉斯迴歸: 0.698
空值補 0, 做羅吉斯迴歸: 0.699
空值補 median, 做羅吉斯迴歸: 0.697


# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [12]:

df['Fare'].fillna(df['Fare'].mode(), inplace=True)
df['Age'].fillna(df['Age'].median(), inplace=True)
train_X = df[:train_num]
estimator = LogisticRegression(solver='lbfgs')
_ = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print(f'原值, 做羅吉斯迴歸: {_:.3f}')

df_mm = MinMaxScaler().fit_transform(df)
train_X = df_mm[:train_num]
estimator = LogisticRegression(solver='lbfgs')
_ = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print(f'最小最大化, 做羅吉斯迴歸: {_:.3f}')

df_sc = StandardScaler().fit_transform(df)
train_X = df_sc[:train_num]
estimator = LogisticRegression(solver='lbfgs')
_ = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print(f'標準化, 做羅吉斯迴歸: {_:.3f}')

原值, 做羅吉斯迴歸: 0.697
最小最大化, 做羅吉斯迴歸: 0.700
標準化, 做羅吉斯迴歸: 0.697
